In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy import ndimage
from scipy.spatial.distance import directed_hausdorff
%matplotlib qt

In [40]:
results = pd.read_csv('../data/evaluation_results/results_29_09_22_08_02_49.csv')

In [41]:
results

,Unnamed: 0,filter,sample,rain,road,mse,mse_db
0,0,imu,sample_id_0,rain_rate_0,s_curve_constant_velocity,8.354667,9.219292
1,1,lidar,sample_id_0,rain_rate_0,s_curve_constant_velocity,10.343207,10.146552
2,2,imu,sample_id_0,rain_rate_10,s_curve_constant_velocity,9.596848,9.821286
3,3,lidar,sample_id_0,rain_rate_10,s_curve_constant_velocity,4.680102,6.702554
4,4,imu,sample_id_0,rain_rate_20,s_curve_constant_velocity,7.590266,8.802570
...,...,...,...,...,...,...,...
187,187,lidar,sample_id_30,rain_rate_10,curve,5.061176,7.042515
188,188,imu,sample_id_30,rain_rate_20,curve,1.542066,1.881030
189,189,lidar,sample_id_30,rain_rate_20,curve,0.847192,-0.720182
190,190,imu,sample_id_30,rain_rate_30,curve,5.718865,7.573098


Bestimmten der unterschiedlichen Filter-typen und Sample Ids

In [42]:
all_samples = results['sample'].unique()
all_filters = results['filter'].unique()
results_sorted_by_samlpes = {}
imu_results_list = []
lidar_results_list = []

Speichern der Daten nach Sample Id und Filtertyp

In [62]:
for sample in all_samples: 
    sample_results = results[results['sample'] == sample]
    imu_results = sample_results[results['filter'] == "imu"]
    lidar_results = sample_results[results['filter'] == "lidar"]   
    imu_mean = np.mean(imu_results['mse'].values)
    lidar_mean = np.mean(lidar_results['mse'].values)
    results_sorted_by_samlpes[sample] = {"imu": imu_mean, "lidar": lidar_mean }
    imu_results_list.append(imu_mean)
    lidar_results_list.append(lidar_mean) 

imu_results_list = np.array(imu_results_list)
lidar_results_list = np.array(lidar_results_list)

c:\Users\mso\Anaconda3\envs\car\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\mso\Anaconda3\envs\car\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Write that shit to csv


In [82]:
data = {'imu': imu_results_list, 'lidar': lidar_results_list}
df = pd.DataFrame(data)
df.to_csv("../data/map/dummy_data.csv", ",")

In [63]:
print("Mean: ", np.array(imu_results_list).mean(),"Median: ", np.array(imu_results_list).median(), " Std: ", np.std(np.array(imu_results_list)))
print("Mean: ", np.array(lidar_results_list).mean(), " Std: ", np.std(np.array(lidar_results_list)))

Mean:  30.090562309511103  Std:  22.483852555950985
Mean:  31.369326610384935  Std:  38.72561607972689


## Auf Normalverteilung prüfen


In [45]:
diff = np.array(imu_results_list) - np.array(lidar_results_list)

In [46]:
shapiro_result = stats.shapiro(diff)


In [47]:
shapiro_result

ShapiroResult(statistic=0.6235021352767944, pvalue=1.1426715218476602e-06)

In [48]:
len(diff)

24

In [69]:
print(ndimage.median(imu_results_list))
print(ndimage.median(lidar_results_list))


24.169548662983626
17.434761403494377


In [64]:
fig, ax = plt.subplots()
ax.set_title("Durchschnittliche RMSE beider Lokalisationen")

ax.boxplot([imu_results_list,lidar_results_list])
ax.set_xticklabels(['Mean RMSE IMU', 'Mean RMSE LIDAR'])

[Text(1, 0, 'Mean RMSE IMU'), Text(2, 0, 'Mean RMSE LIDAR')]

In [78]:
wilcoxon_result = stats.wilcoxon(x=imu_results_list, y=lidar_results_list, alternative="greater")

In [79]:
wilcoxon_result

WilcoxonResult(statistic=785.0, pvalue=0.021647218043793638)

In [ ]:
data 